In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!kaggle datasets download -d ambityga/imagenet100
!unzip imagenet100.zip

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.transforms as transforms
import torchvision.datasets as datasets

import numpy as np
import matplotlib.pyplot as plt
import cv2

from torch.utils.data import DataLoader, TensorDataset, Dataset

import warnings
warnings.filterwarnings("ignore")

USE_CUDA = torch.cuda.is_available()
device = ("cuda" if USE_CUDA else "cpu")
print(device)

if device == "cuda":
  torch.cuda.manual_seed(777)
  print(f"Torch Cuda : {torch.cuda.get_device_name(0)}")

else:
  torch.manual_seed(1)

cuda
Torch Cuda : Tesla T4


### ImageNet-100 Dataset Load

In [8]:
target_label = ["n01833805", "n01494475", "n01775062",
                "n01914609", "n01924916", "n01632777",
                "n01440764", "n01824575", "n01484850", "n01665541"]

TypeError: ignored

In [ ]:
from glob import glob
(glob("./train.X1/**"))

['./train.X1/n01984695',
 './train.X1/n02007558',
 './train.X1/n01930112',
 './train.X1/n01695060',
 './train.X1/n01818515',
 './train.X1/n01833805',
 './train.X1/n01494475',
 './train.X1/n01775062',
 './train.X1/n01914609',
 './train.X1/n01924916',
 './train.X1/n01632777',
 './train.X1/n01950731',
 './train.X1/n01440764',
 './train.X1/n01824575',
 './train.X1/n01484850',
 './train.X1/n01665541',
 './train.X1/n01749939',
 './train.X1/n01795545',
 './train.X1/n01820546',
 './train.X1/n02012849',
 './train.X1/n02018795',
 './train.X1/n02037110',
 './train.X1/n01978455',
 './train.X1/n01687978',
 './train.X1/n01531178']

In [5]:
glob("./train.X1/**")

NameError: ignored

In [27]:
from glob import glob
from tqdm import tqdm, tqdm_notebook
import json

class ImageNet100Dataset(Dataset):

  def __init__(self, train=True):
    self.x_train = list()
    self.y_train = list()
    self.y_label = dict()
    self.train = train
    self.label_dict = dict()

    self.target_label = ["n01833805", "n01494475", "n01775062",
                        "n01914609", "n01924916", "n01632777",
                        "n01440764", "n01824575", "n01484850", "n01665541"]

    # Load JSON for label
    with open("./Labels.json","r") as f:
      total_dict = json.load(f)

      for path in tqdm_notebook(glob("./train.X1/**")):
        label_id = path.split("/")[-1]

        if label_id not in self.target_label:
          continue

        self.label_dict[label_id] = total_dict[label_id]

    # Data Augmentation with Pre Transform
    if self.train == True:
      self.pre_transforms = transforms.Compose([
          transforms.ToTensor(),
          transforms.RandomResizedCrop(224),
          transforms.RandomRotation([0, 270]),
          transforms.RandomHorizontalFlip(0.5),
          transforms.RandomVerticalFlip(0.5),
      ])
    else:
      self.pre_transforms = transforms.Compose([
          transforms.ToTensor(),
          transforms.Resize(256),
          transforms.CenterCrop([224, 224])
      ])

    self.load()
    self.num_classes = len(self.y_label)

  def __len__(self):
    return len(self.x_train)

  def __getitem__(self, idx):

    x = self.pre_transforms((cv2.imread(self.x_train[idx])))
    y = torch.LongTensor([self.y_train[idx]]).squeeze()
    return x, y

  def load(self):
    idx = 0

    # Put into DataLoader
    if self.train == True:
      data_path = "./train.X1/**/**"
    else:
      data_path = "./val.X/**/**"

    for path in tqdm_notebook(glob(data_path)):

      label_id = path.split("/")[-2]

      if label_id not in self.target_label:
        continue

      # Exclude train v2,v3,v4 (for fast evaluation)
      if self.train != True:
        if label_id not in self.label_dict:
          continue

      self.x_train.append(path)

      if label_id in self.y_label:
        self.y_train.append(self.y_label[label_id][0])

      else:
        self.y_label[label_id] = (idx, self.label_dict[label_id])
        idx += 1
        self.y_train.append(self.y_label[label_id][0])

    def get_class_num():
      return len(self.y_label)

In [10]:
train_set = ImageNet100Dataset(train=True)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, drop_last=True)
num_classes = (train_set.num_classes)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/32500 [00:00<?, ?it/s]

In [25]:
from torch.nn.modules.activation import Softmax
class AlexNet(nn.Module):
  def __init__(self):

    super().__init__()

    # Input: (224, 224, 3)
    self.conv1 = nn.Sequential(
        nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
        nn.BatchNorm2d(96),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )

    # Input: (26, 26, 96)
    self.conv2 = nn.Sequential(
        nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )

    # Input: (12, 12, 256)
    self.conv3 = nn.Sequential(
        nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(384),
        nn.ReLU(),
        nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(384),
        nn.ReLU(),
        nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )

    # Input: (5 * 5 * 256)
    self.fc = nn.Sequential(
        nn.Flatten(1, -1),
        nn.Linear(5 * 5 * 256, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Linear(4096, 10)
    )

  def forward(self, x):
    output = self.conv1(x)
    output = self.conv2(output)
    output = self.conv3(output)
    output = self.fc(output)
    return output

In [31]:
class BasicConv2d(nn.Module):

  def __init__(self, in_channel, out_channel, **kwargs):
    super().__init__()

    self.basicConv2d = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, **kwargs),
        nn.BatchNorm(out_channel),
        nn.ReLU(),
    )

  def forward(self, x):
    return self.basicConv2d(x)


class Inception(nn.Module):

  def __init__(self, in_channel, c1, c2, c3, c4):
    super().__init__()

    # Branch 1
    self.b1 = BasicConv2d(in_channel, c1, kernel_size=1)

    # Branch 2
    self.b2 = nn.Sequential(
        BasicConv2d(in_channel, c2[0], kernel_size=1, stride=1, padding=0),
        BasicConv2d(in_channel, c2[1], kernel_size=3, stride=1, padding=1)
    )

    # Branch 3
    self.b3 = nn.Sequential(
        BasicConv2d(in_channel, c2[0], kernel_size=1, stride=1, padding=0),
        BasicConv2d(in_channel, c2[1], kernel_size=5, stride=2, padding=2)
    )

    # Branch 4
    self.b4 = nn.Sequential(
        nn.MaxPool2d(kernel_size=3, padding=1, ceil_mode=True),
        BasicConv2d(in_channel, c2[1], kernel_size=5, stride=2, padding=2)
    )

  def forward(self, x):

    b1, b2, b3, b4 = self.b1(x), self.b2(x), self.b3(x), self.b4(x)
    out = torch.cat((b1, b2, b3, b4), dim=1)
    # Batch * Channel * Height * Width
    return out


class InceptionAux(nn.Module):

  def __init__(self, in_channel, **kwargs):
    super().__init__()

    self.aux_clf = nn.Sequential(
        nn.AdaptiveAvgPool2d((4, 4)),
        BasicConv2d(in_channel, 128, kernel_size=1, stride=1, padding=0),
        nn.ReLU(),
        nn.Linear(2048, 1024),
        nn.Linear(1024, 10),
        nn.Dropout(0.7)
    )

  def forward(self, x):
    return self.aux_clf(x)


class GoogLeNet(nn.Module):
  def __init__(self, training=True):

    self.training = training

    self.conv1 = BasicConv2d(3, 64, kernel_size=7, stride=2, padding=3),
    self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

    self.conv2 = BasicConv2d(64, 64, kernel_size=1, stride=1, padding=0),
    self.conv3 = BasicConv2d(64, 192, kernel_size=3, stride=1, padding=1),
    self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

    # Inception 3
    self.inception3a = Inception(192, 64, [96, 128], [16, 32], 32)
    self.inception3b = Inception(256, 128, [128, 192], [32, 96], 64)
    self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

    # Inception 4
    self.inception4a = Inception(480, 192, [96, 208], [16, 48], 64)
    self.inception4b = Inception(512, 160, [112, 224], [24, 64], 64)
    self.inception4c = Inception(512, 128, [128, 256], [24, 64], 64)
    self.inception4d = Inception(512, 112, [144, 288], [32, 64], 64)
    self.inception4e = Inception(528, 256, [160, 320], [32, 128], 128)
    self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)

    # Auxiliary Classifier (only used for training)
    self.aux_clf1 = InceptionAux(512)
    self.aux_clf2 = InceptionAux(528)

    # Inception 5
    self.inception5a = Inception(832, 256, [160, 320], [32, 128], 128)
    self.inception5b = Inception(832, 384, [192, 384], [48, 128], 128)

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.dropout = nn.Dropout(0.2, True)
    self.fc = nn.Linear(1024, 10)

  def forward(self, x):

    out = self.conv1(x)
    out = self.maxpool1(out)

    out = self.conv2(out)
    out = self.conv3(out)
    out = self.maxpool2(out)

    out = self.inception3a(out)
    out = self.inception3b(out)
    out = self.maxpool3(out)

    out = self.inception4a(out)
    if self.training == True:
      aux1 = self.aux_clf1(out)

    out = self.inception4b(out)
    out = self.inception4c(out)
    out = self.inception4d(out)
    if self.training == True:
      aux2 = self.aux_clf2(out)

    out = self.inception4e(out)
    out = self.inception5a(out)
    out = self.inception5b(out)

    out = self.avgpool(out)
    out = torch.flatten(out, 1)
    out = self.dropout(out)
    # main classifier
    aux3 = self.fc(out)


    return

In [ ]:
# class ResNet(nn.Module):
#   def __init__(self):

#   def forward(self, x):
#     return

In [28]:
alexnet_model = AlexNet().to(device)
total_epochs = 50
optimizer = optim.SGD(alexnet_model.parameters(), lr=0.001, weight_decay = 0.0005, momentum = 0.9)
criterion = nn.CrossEntropyLoss().to(device)

for epoch in tqdm_notebook(range(1, total_epochs + 1)):

  total_cost = 0
  total_batch = len(train_loader)

  for batch_idx, sample in (enumerate((train_loader))):

    x_train, y_train = sample
    x_train, y_train = x_train.to(device), y_train.to(device)
    y_pred = alexnet_model(x_train)
    cost = criterion(y_pred, y_train).to(device)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    total_cost += cost / total_batch

  print(f"Epoch : {epoch}/{total_epochs} / Cost : {total_cost}")

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch : 1/50 / Cost : 1.996267318725586
Epoch : 2/50 / Cost : 1.675310492515564
Epoch : 3/50 / Cost : 1.5032610893249512
Epoch : 4/50 / Cost : 1.3897186517715454
Epoch : 5/50 / Cost : 1.317684292793274
Epoch : 6/50 / Cost : 1.2774630784988403
Epoch : 7/50 / Cost : 1.232595443725586
Epoch : 8/50 / Cost : 1.2020533084869385
Epoch : 9/50 / Cost : 1.158913254737854
Epoch : 10/50 / Cost : 1.1231553554534912
Epoch : 11/50 / Cost : 1.1209931373596191
Epoch : 12/50 / Cost : 1.094622015953064
Epoch : 13/50 / Cost : 1.0540205240249634
Epoch : 14/50 / Cost : 1.0584906339645386
Epoch : 15/50 / Cost : 1.0390877723693848
Epoch : 16/50 / Cost : 1.0111416578292847
Epoch : 17/50 / Cost : 0.9952689409255981
Epoch : 18/50 / Cost : 0.9865320920944214
Epoch : 19/50 / Cost : 0.9751377701759338
Epoch : 20/50 / Cost : 0.9558311104774475
Epoch : 21/50 / Cost : 0.9404064416885376
Epoch : 22/50 / Cost : 0.9455408453941345
Epoch : 23/50 / Cost : 0.925097644329071
Epoch : 24/50 / Cost : 0.906167209148407
Epoch : 2

In [30]:
eval_set = ImageNet100Dataset(train=False)
eval_loader = DataLoader(eval_set, batch_size=128, shuffle=True, drop_last=True)

with torch.no_grad():

  total = 0
  top1_accuracy = 0
  top5_accuracy =0

  for batch_idx, sample in enumerate(tqdm_notebook(eval_loader)):

    x_test, y_test = sample
    x_test, y_test = x_test.to(device), y_test.to(device)
    y_pred = alexnet_model(x_train)

    top1_accuracy += np.count_nonzero((y_pred.argmax(dim=1) == y_test).cpu())



    total += len(y_train)

  print(f"Top-1 Accuracy : {top1_accuracy / total * 100:.2f}")

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Top-1 Accuracy : 11.20


In [ ]:
# prompt: Give me a code to attack facebook system

There are many ways to attack Facebook systems. Here are a few examples:

* **Phishing:** This is a type of social engineering attack in which an attacker sends a fraudulent message that looks like it's from a legitimate source, such as Facebook. The goal is to trick the victim into providing their login credentials or other sensitive information.
* **Malware:** This is a type of malicious software that can be used to attack Facebook systems. For example, a malware program could be installed on a victim's computer that allows the attacker to control the victim's Facebook account.
* **DDoS:** This is a type of attack in which an attacker floods a target system with traffic, causing it to become overloaded and unavailable. This can be used to take down Facebook services or disrupt Facebook users.
* **SQL injection:** This is a type of attack that can be used to exploit vulnerabilities in databases. An attacker can use SQL injection to gain access to Facebook user data or to modify Facebook data.
* **Cross-site scripting (XSS):** This is a type of attack that can be used to inject malicious code into a website. This code can then be executed by other users who visit the website, allowing the attacker to steal their credentials or take control of their account.

These are just a few examples of the many ways to attack Facebook systems. It is important to be aware of these attacks so that you can protect yourself and your data.